**1.Dataset Statistics**

In [ ]:
import pandas as pd

# -----------------------------------------------------------------------------
# Load the tourism dataset from 'tourism.csv'
# The CSV file is assumed to have:
# - An index column with no header (starting from 1)
# - The following columns with headers: Quarter, Region, State, Purpose, Trips
# -----------------------------------------------------------------------------

# Read the CSV file, using the first column as the index
tourism = pd.read_csv('datasets/tourism.csv', index_col=0)

# Display the first few rows to verify the data
print("First few rows of the dataset:")
print(tourism.head())

# -----------------------------------------------------------------------------
# 1. Compute the Mean Number of Trips for Each Group and Sort by Mean
# -----------------------------------------------------------------------------

# Group by Region, State, and Purpose, then compute the mean of Trips
grouped_mean = tourism.groupby(['Region', 'State', 'Purpose'], as_index=False)['Trips'].mean()

# Sort the groups by the mean number of trips
grouped_mean_sorted = grouped_mean.sort_values('Trips')

print("\nMean of Trips for each group (sorted by mean):")
print(grouped_mean_sorted.head(10))

# -----------------------------------------------------------------------------
# 2. Compute a Five-Number Summary for Each Group
#    (Minimum, 25th percentile, Median, 75th percentile, and Maximum)
# -----------------------------------------------------------------------------

# Define a function to compute the five-number summary using quantiles
summary_stats = tourism.groupby(['Region', 'State', 'Purpose'])['Trips'].agg(
    min=lambda x: x.quantile(0),
    Q1=lambda x: x.quantile(0.25),
    median='median',
    Q3=lambda x: x.quantile(0.75),
    max=lambda x: x.quantile(1)
).reset_index()

print("\nSummary statistics for each group:")
print(summary_stats.head(10))

**2.ACF Features**

In [ ]:
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import acf

# ---------------------------------------------------------------------------
# Function to compute ACF-based features for a given time series.
# ---------------------------------------------------------------------------
def compute_acf_features(series, lags=10, seasonal_period=None):
    """
    Compute autocorrelation features for a time series.
    
    Parameters:
      series : pandas Series
          The time series data.
      lags : int
          The number of lags to use when computing the sum of squares.
      seasonal_period : int or None
          The seasonal lag (e.g., 4 for quarterly data). If provided, 
          the autocorrelation at this lag is computed.
    
    Returns:
      A pandas Series with the following features:
        - acf1: First autocorrelation coefficient of the original series.
        - acf10: Sum of squares of the first 10 autocorrelation coefficients.
        - diff1_acf1: First autocorrelation coefficient of the first-differenced series.
        - diff1_acf10: Sum of squares of the first 10 autocorrelation coefficients of the first-differenced series.
        - diff2_acf1: First autocorrelation coefficient of the twice-differenced series.
        - diff2_acf10: Sum of squares of the first 10 autocorrelation coefficients of the twice-differenced series.
        - season_acf1: Autocorrelation coefficient at the seasonal lag (if seasonal_period is provided).
    """
    # Compute ACF for the original series (nlags includes lag 0)
    acf_vals = acf(series, nlags=lags, fft=False)
    acf1 = acf_vals[1] if len(acf_vals) > 1 else np.nan
    acf10 = np.sum(acf_vals[1:lags+1] ** 2) if len(acf_vals) > lags else np.nan

    # First difference
    diff1 = series.diff().dropna()
    acf_diff1 = acf(diff1, nlags=lags, fft=False)
    diff1_acf1 = acf_diff1[1] if len(acf_diff1) > 1 else np.nan
    diff1_acf10 = np.sum(acf_diff1[1:lags+1] ** 2) if len(acf_diff1) > lags else np.nan

    # Second difference
    diff2 = diff1.diff().dropna()
    acf_diff2 = acf(diff2, nlags=lags, fft=False)
    diff2_acf1 = acf_diff2[1] if len(acf_diff2) > 1 else np.nan
    diff2_acf10 = np.sum(acf_diff2[1:lags+1] ** 2) if len(acf_diff2) > lags else np.nan

    # Seasonal autocorrelation at the given seasonal period (if provided)
    if seasonal_period is not None and seasonal_period < len(acf_vals):
        season_acf1 = acf_vals[seasonal_period]
    else:
        season_acf1 = np.nan

    return pd.Series({
        'acf1': acf1,
        'acf10': acf10,
        'diff1_acf1': diff1_acf1,
        'diff1_acf10': diff1_acf10,
        'diff2_acf1': diff2_acf1,
        'diff2_acf10': diff2_acf10,
        'season_acf1': season_acf1
    })

# ---------------------------------------------------------------------------
# Load the tourism dataset from tourism.csv.
# The CSV file is assumed to have:
# - An index column with no header (starting from 1)
# - The columns: Quarter, Region, State, Purpose, Trips
# ---------------------------------------------------------------------------

# Read the CSV file, using the first column as the index.
tourism = pd.read_csv('datasets/tourism.csv', index_col=0)

# Display the first few rows to verify the data
print("First few rows of the dataset:")
print(tourism.head())

# ---------------------------------------------------------------------------
# Compute ACF features for each group (grouped by Region, State, and Purpose).
# ---------------------------------------------------------------------------
# Here we apply the compute_acf_features function to the 'Trips' column of each group.
# For seasonal data, we assume a seasonal period of 4 (e.g., quarterly data).

acf_features = tourism.groupby(['Region', 'State', 'Purpose'])['Trips'] \
    .apply(lambda x: compute_acf_features(x, lags=10, seasonal_period=4))

# Reset index to turn the group keys into columns.
acf_features = acf_features.reset_index()

# Display the computed ACF features for the first 10 groups
print("\nACF features for the first 10 groups:")
print(acf_features.head(10))

**3.STL Features**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import STL

%matplotlib inline
sns.set(style="whitegrid")

# -----------------------------------------------------------------------------
# 1. Load and Prepare the Tourism Data
# -----------------------------------------------------------------------------
# Assume the CSV file 'tourism.csv' has:
# - An index column with no header (starting from 1)
# - Columns: Quarter, Region, State, Purpose, Trips

tourism = pd.read_csv('datasets/tourism.csv', index_col=0)

# Convert Quarter to a PeriodIndex (assume format like '1998-Q1')
tourism['Quarter'] = pd.PeriodIndex(tourism['Quarter'], freq='Q')

# Sort the data by Quarter
tourism = tourism.sort_values('Quarter')

print("First few rows of the dataset:")
print(tourism.head())

# -----------------------------------------------------------------------------
# 2. Define a Function to Compute STL-based Features
# -----------------------------------------------------------------------------
def compute_stl_features(series, period=4):
    """
    Decompose a time series using STL (additive) and compute:
      - trend_strength = max(0, 1 - Var(R) / Var(T + R))
      - seasonal_strength_year = max(0, 1 - Var(R) / Var(S + R))
      
    Parameters:
      series : pandas Series (the time series, indexed by time)
      period : int, the seasonal period (4 for quarterly data)
      
    Returns:
      A pandas Series with the computed features.
    """
    # Ensure the series is sorted by time
    series = series.sort_index()
    
    # Perform STL decomposition
    stl = STL(series, period=period, robust=True)
    result = stl.fit()
    
    # Extract components
    T = result.trend
    S = result.seasonal
    R = result.resid
    
    # Compute sample variances (using ddof=1 for unbiased estimate)
    var_R = np.var(R, ddof=1)
    var_T_plus_R = np.var(T + R, ddof=1)
    var_S_plus_R = np.var(S + R, ddof=1)
    
    # Compute strengths (ensuring a minimum value of 0)
    trend_strength = max(0, 1 - var_R / var_T_plus_R) if var_T_plus_R > 0 else np.nan
    seasonal_strength_year = max(0, 1 - var_R / var_S_plus_R) if var_S_plus_R > 0 else np.nan
    
    return pd.Series({'trend_strength': trend_strength,
                      'seasonal_strength_year': seasonal_strength_year})

# -----------------------------------------------------------------------------
# 3. Compute STL Features for Each Group and Pivot the Results
# -----------------------------------------------------------------------------
# Group by Region, State, and Purpose and compute the features on the 'Trips' series.
features = tourism.groupby(['Region', 'State', 'Purpose'])['Trips'] \
                  .apply(lambda x: compute_stl_features(x, period=4))

# Reset the index so that the computed feature names are in a column "level_3" and their values in "Trips"
features_df = features.reset_index()

# Pivot the DataFrame so each feature becomes a separate column.
features_df = features_df.pivot_table(index=["Region", "State", "Purpose"],
                                        columns="level_3", values="Trips").reset_index()

print("\nSTL-based features (first 10 rows):")
print(features_df.head(10))

# -----------------------------------------------------------------------------
# 4. Visualization 1: Scatter Plot of Trend Strength vs. Seasonal Strength
# -----------------------------------------------------------------------------
# Create a scatter plot of trend_strength vs seasonal_strength_year, colored by Purpose,
# and faceted by State.
g = sns.FacetGrid(features_df, col="State", hue="Purpose", col_wrap=3, height=4, sharex=False, sharey=False)
g.map_dataframe(plt.scatter, "trend_strength", "seasonal_strength_year", edgecolor="w")
g.add_legend()
g.fig.suptitle("Trend Strength vs Seasonal Strength (STL Features)", y=1.03)
plt.show()

# -----------------------------------------------------------------------------
# 5. Visualization 2: Plot the Most Seasonal Series
# -----------------------------------------------------------------------------
# Identify the group(s) with the highest seasonal strength.
max_season_strength = features_df['seasonal_strength_year'].max()
most_seasonal = features_df[features_df['seasonal_strength_year'] == max_season_strength]
print("\nGroup(s) with highest seasonal strength:")
print(most_seasonal)

# Merge these groups back with the original tourism data to obtain their full time series.
most_seasonal_data = pd.merge(tourism, most_seasonal, on=['Region', 'State', 'Purpose'], how='inner')

# For plotting, convert the Quarter Period to a string (but we will hide the labels to reduce clutter).
most_seasonal_data['Quarter_str'] = most_seasonal_data['Quarter'].astype(str)

# Use FacetGrid to create line plots for these most seasonal series.
g2 = sns.FacetGrid(most_seasonal_data, row="State", col="Region", hue="Purpose", height=3, aspect=1.5, margin_titles=True)
g2.map_dataframe(plt.plot, "Quarter_str", "Trips")
g2.add_legend()
g2.fig.suptitle("Time Series for the Most Seasonal Tourism Series", y=1.02)

# Hide x-axis tick labels to avoid clutter
for ax in g2.axes.flatten():
    ax.set_xticklabels([])

plt.show()